In [ ]:
# NOTES
# Correct! Here is a complete list of columns that hold alternate title data: Also known as, Arabic, Cantonese, Chinese, 
# French, Hangul, Hebrew, Hepburn, Japanese, Literally,  Mandarin, McCune–Reischauer, Original title, Polish,  
# Revised Romanization, Romanized, Russian, Simplified, Traditional, Yiddish

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
file_dir = 'C:\\Users\kbead\Data Boot Camp\Modules\Module 8'

In [ ]:
with open(f'{file_dir}\wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
len(wiki_movies_raw)

In [ ]:
wiki_movies_raw[3600:3605]

In [ ]:
# read csv files using pandas
kaggle_metadata = pd.read_csv(f'{file_dir}\\movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}\\ratings.csv')

In [ ]:
# Skill drill 8.2.2 practice use of sample() method. Fora DataFrame called df, df.sample(n=5) will show five random rows
kaggle_metadata.sample(n=5)

In [ ]:
ratings.sample(n=10)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

In [ ]:
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie) 
                   and 'imdb_link' in movie 
                   and 'No. of episodes' not in movie]
len(wiki_movies)

In [ ]:
# wiki_movies_df = pd.DataFrame(wiki_movies)
# wiki_movies_df.head()

In [ ]:
def clean_movie(movie):
    movie = dict(movie) # <--create a non-destructive copy
    
    
    alt_titles = {}     # <--create empy dict to later populate
    
    # add list of alt keys to loop through
    for key in ['Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally', 'Mandarin', 
                'McCune–Reischauer', 'Original title', 'Polish', 'Revised Romanization', 'Romanized', 'Russian', 'Simplified', 
                'Traditional', 'Yiddish']:
        
          
        if key in movie:                   # <--check if key is in movie object 
            alt_titles[key] = movie[key]   # <--add the alt titles to the dict
            movie.pop(key)                 # <--remove the key-value pair
            
    if len(alt_titles) > 0:              # <--make sure the dict has key-value pairs
        movie['alt_titles'] = alt_titles # <--add the dict to the movie object
        
    return movie 

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
# use list comprehension to make a list of cleaned movies
clean_movies = [clean_movie(movie) for movie in wiki_movies]

# set the wiki_movies_df to be the DataFrame created from clean_movies
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())